## Medium Article Code

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
num_epochs = 5
batch_size = 40
learning_rate = 0.001
classes = ('plane', 'car' , 'bird',
    'cat', 'deer', 'dog',
    'frog', 'horse', 'ship', 'truck')

cuda


In [ ]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
    )
])
train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)

100%|██████████| 170M/170M [00:04<00:00, 35.1MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset
    , batch_size = batch_size
    , shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset
    , batch_size = batch_size
    , shuffle = True)
n_total_step = len(train_loader)
print(n_total_step)

1250


In [ ]:
model = models.vgg16(pretrained = True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(input_lastLayer,10)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 128MB/s]


In [ ]:
for epoch in range(num_epochs):
    for i, (imgs , labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        labels_hat = model(imgs)
        n_corrects = (labels_hat.argmax(axis=1)==labels).sum().item()
        loss_value = criterion(labels_hat, labels)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (i+1) % 250 == 0:
          print(f"epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {loss_value:.5f}, acc = {100*(n_corrects/labels.size(0)):.2f}%")
    print()



























































































































































































































































epoch 1/5, step: 250/1250: loss = 0.28140, acc = 87.50%


























































































































































































































































epoch 1/5, step: 500/1250: loss = 0.27473, acc = 85.00%


























































































































































































































































epoch 1/5, step: 750/1250: loss = 0.23134, acc = 92.50%





















































































KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    number_corrects = 0
    number_samples = 0
    for i, (test_images_set , test_labels_set) in enumerate(test_loader):
        test_images_set = test_images_set.to(device)
        test_labels_set = test_labels_set.to(device)
        y_predicted = model(test_images_set)
        labels_predicted = y_predicted.argmax(axis = 1)
        number_corrects += (labels_predicted==test_labels_set).sum().item()
        number_samples += test_labels_set.size(0)
    print(f'Overall accuracy {(number_corrects / number_samples)*100}%')

Overall accuracy 91.33%


In [ ]:
heatmap = pd.DataFrame(data=0,index=classes,columns=classes)
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(batch_size):
            true_label = labels[i].item()
            predicted_label = predicted[i].item()
            heatmap.iloc[true_label,predicted_label] += 1
_, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(heatmap, annot=True, fmt=”d”,cmap=”YlGnBu”)
plt.show()

In [ ]:
!pip install codecarbon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.2/504.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.4/386.4 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0
  Attempting uninstall: prompt_toolkit
    Found existing installation: prompt_toolkit 3.0.48
    Uninstalling prompt_toolkit-3.0.48:
      Successfully uninstalled prompt_toolkit-3.0.48
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ip

In [ ]:
!pip install thop

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets, transforms
import time
import os
import subprocess
import numpy as np
from tqdm import tqdm
import codecarbon
from thop import profile

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Define the transforms for the dataset with additional augmentations
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 dataset
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split the full training dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Create DataLoaders for the train, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Print dataset sizes for debugging
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

100%|██████████| 170M/170M [00:05<00:00, 28.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Training set size: 40000
Validation set size: 10000
Test set size: 10000


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Load the pretrained VGG16 model and modify the classifier
model = models.vgg16(pretrained=True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Sequential(
    nn.Dropout(0.5),  # Increased dropout for regularization
    nn.Linear(input_lastLayer, 10)
)
model = model.to(device)

# Define loss function and optimizer with learning rate scheduler
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Tracking metrics for analysis
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Use CodeCarbon for system-wide energy consumption tracking
tracker = codecarbon.EmissionsTracker()
tracker.start()

# Function to get GPU power consumption using nvidia-smi
def get_gpu_power():
    try:
        # Query power.draw from nvidia-smi
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=power.draw', '--format=csv,nounits,noheader'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        power_draws = result.stdout.strip().split('\n')
        power_draws = [float(p) for p in power_draws]
        avg_power_draw = sum(power_draws) / len(power_draws)  # Average power across GPUs
        return avg_power_draw  # in watts
    except Exception as e:
        print(f"Error getting GPU power: {e}")
        return 0.0

# Tracking GPU-specific energy consumption
gpu_power_readings = []

# Training loop with GPU power monitoring
epochs = 25
start_training_time = time.time()

for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Record GPU power usage
        gpu_power_readings.append(get_gpu_power())

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Append metrics for plotting
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    # Append metrics for plotting
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Epoch [{epoch+1}/{epochs}], Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)

training_time = time.time() - start_training_time

# Stop the CodeCarbon tracker after training
emissions = tracker.stop()

# Calculate average GPU power and total GPU energy consumption (in kWh)
avg_gpu_power = sum(gpu_power_readings) / len(gpu_power_readings) if gpu_power_readings else 0.0
energy_consumption_gpu = (avg_gpu_power * training_time) / 3600  # Convert to kWh

# Save the model after training
model_path = "trained_vgg16_model.pth"
torch.save(model.state_dict(), model_path)

# Model analysis metrics
param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
torch.save(model.state_dict(), "temp_model.pth")
model_size = os.path.getsize("temp_model.pth") / (1024 * 1024)  # Convert bytes to MB
os.remove("temp_model.pth")

# FLOPs calculation
dummy_input = torch.randn(1, 3, 32, 32).to(device)
flops, _ = profile(model, inputs=(dummy_input,), verbose=False)

# Inference time calculation
model.eval()
inference_start = time.time()
with torch.no_grad():
    for _ in range(100):
        _ = model(dummy_input)
inference_time = (time.time() - inference_start) / 100

# Final energy and emissions report
print("\n--- Model Analysis ---")
print(f"Parameter Count: {param_count}")
print(f"Model Size: {model_size:.2f} MB")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Average Inference Time: {inference_time:.6f} seconds")

print("\n--- Energy and Emissions Report ---")
print(f"CO2 Emissions (CodeCarbon): {emissions:.6f} kg")
print(f"Average GPU Power Consumption: {avg_gpu_power:.2f} W")
print(f"Total GPU Energy Consumption: {energy_consumption_gpu:.6f} kWh")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 86.1MB/s]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[codecarbon 

Epoch [1/25], Train Loss: 1.2619, Train Accuracy: 57.04%
Epoch [1/25], Val Loss: 0.9031, Val Accuracy: 69.01%


Epoch 2/25 - Training:   0%|          | 0/625 [00:00<?, ?it/s][codecarbon INFO @ 03:25:56] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:25:56] Energy consumed for all GPUs : 0.001179 kWh. Total GPU Power : 45.910643096133235 W
[codecarbon INFO @ 03:25:56] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:25:56] 0.002163 kWh of electricity used since the beginning.
Epoch 2/25 - Training:  23%|██▎       | 143/625 [00:15<00:44, 10.86it/s][codecarbon INFO @ 03:26:11] Energy consumed for RAM : 0.000119 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:26:11] Energy consumed for all GPUs : 0.001431 kWh. Total GPU Power : 60.52333402443282 W
[codecarbon INFO @ 03:26:11] Energy consumed for all CPUs : 0.001063 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:26:11] 0.002612 kWh of electricity used since the beginning.
Epoch 2/25 - Training:  46%|████▌     | 285/625 [00:29<00:30, 10.97it/s][

Epoch [2/25], Train Loss: 0.8249, Train Accuracy: 73.43%
Epoch [2/25], Val Loss: 0.8351, Val Accuracy: 72.66%


Epoch 3/25 - Training:   1%|▏         | 9/625 [00:01<01:06,  9.33it/s][codecarbon INFO @ 03:27:11] Energy consumed for RAM : 0.000198 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:27:11] Energy consumed for all GPUs : 0.002399 kWh. Total GPU Power : 46.33225168392685 W
[codecarbon INFO @ 03:27:11] Energy consumed for all CPUs : 0.001771 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:27:11] 0.004368 kWh of electricity used since the beginning.
Epoch 3/25 - Training:  26%|██▌       | 160/625 [00:16<00:40, 11.34it/s][codecarbon INFO @ 03:27:26] Energy consumed for RAM : 0.000218 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:27:26] Energy consumed for all GPUs : 0.002660 kWh. Total GPU Power : 62.68205276766801 W
[codecarbon INFO @ 03:27:26] Energy consumed for all CPUs : 0.001948 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:27:26] 0.004826 kWh of electricity used since the beginning.
Epoch 3/25 - Training:  50%|█████     | 313/625 [00:31<00:28, 11.0

...

Epoch [23/25], Train Loss: 0.1905, Train Accuracy: 93.61%
Epoch [23/25], Val Loss: 0.3435, Val Accuracy: 89.07%


Epoch 24/25 - Training:   6%|▌         | 36/625 [00:03<00:53, 11.05it/s][codecarbon INFO @ 03:52:42] Energy consumed for RAM : 0.002216 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:52:42] Energy consumed for all GPUs : 0.028608 kWh. Total GPU Power : 49.008633671762794 W
[codecarbon INFO @ 03:52:42] Energy consumed for all CPUs : 0.019838 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:52:42] 0.050663 kWh of electricity used since the beginning.
[codecarbon INFO @ 03:52:42] 0.010518 g.CO2eq/s mean an estimation of 331.68648771941275 kg.CO2eq/year
Epoch 24/25 - Training:  30%|██▉       | 185/625 [00:18<00:41, 10.55it/s][codecarbon INFO @ 03:52:57] Energy consumed for RAM : 0.002236 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:52:57] Energy consumed for all GPUs : 0.028878 kWh. Total GPU Power : 64.74954774069738 W
[codecarbon INFO @ 03:52:57] Energy consumed for all CPUs : 0.020015 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:52:57] 0.051129 kWh 

Epoch [24/25], Train Loss: 0.1556, Train Accuracy: 94.83%
Epoch [24/25], Val Loss: 0.3365, Val Accuracy: 89.24%


Epoch 25/25 - Training:   8%|▊         | 51/625 [00:05<00:52, 11.01it/s][codecarbon INFO @ 03:53:57] Energy consumed for RAM : 0.002315 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:53:57] Energy consumed for all GPUs : 0.029894 kWh. Total GPU Power : 49.79539689484897 W
[codecarbon INFO @ 03:53:57] Energy consumed for all CPUs : 0.020724 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:53:57] 0.052934 kWh of electricity used since the beginning.
Epoch 25/25 - Training:  32%|███▏      | 200/625 [00:20<00:37, 11.27it/s][codecarbon INFO @ 03:54:12] Energy consumed for RAM : 0.002335 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:54:12] Energy consumed for all GPUs : 0.030165 kWh. Total GPU Power : 64.99333825412349 W
[codecarbon INFO @ 03:54:12] Energy consumed for all CPUs : 0.020901 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:54:12] 0.053402 kWh of electricity used since the beginning.
Epoch 25/25 - Training:  56%|█████▌    | 351/625 [00:35<00:24, 

Epoch [25/25], Train Loss: 0.1442, Train Accuracy: 95.14%
Epoch [25/25], Val Loss: 0.3299, Val Accuracy: 89.41%

--- Model Analysis ---
Parameter Count: 134301514
Model Size: 512.33 MB
FLOPs: 0.43 GFLOPs
Training Time: 1824.55 seconds
Average Inference Time: 0.003761 seconds

--- Energy and Emissions Report ---
CO2 Emissions (CodeCarbon): 0.019207 kg
Average GPU Power Consumption: 63.74 W
Total GPU Energy Consumption: 32.306925 kWh


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
from thop import profile
from codecarbon import EmissionsTracker
import time
import os
import subprocess

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the transforms for the dataset
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 dataset
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split the full training dataset into training and validation sets
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Create DataLoaders for the train, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Define the teacher model (VGG16) and load pre-trained weights
teacher_model = models.vgg16(pretrained=False)
teacher_last_layer = teacher_model.classifier[6].in_features
teacher_model.classifier[6] = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(teacher_last_layer, 10)
)
teacher_model.load_state_dict(torch.load("trained_vgg16_model.pth"))  # Load the downloaded teacher model
teacher_model = teacher_model.to(device)
teacher_model.eval()  # Set the teacher model to evaluation mode

# Define the student model (VGG11)
student_model = models.vgg11(pretrained=False)
student_last_layer = student_model.classifier[6].in_features
student_model.classifier[6] = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(student_last_layer, 10)
)
student_model = student_model.to(device)

# Define the distillation loss
class DistillationLoss(nn.Module):
    def __init__(self, temperature=3.0, alpha=0.5):
        super(DistillationLoss, self).__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.criterion_ce = nn.CrossEntropyLoss()

    def forward(self, student_logits, teacher_logits, labels):
        distillation_loss = nn.KLDivLoss(reduction='batchmean')(
            torch.log_softmax(student_logits / self.temperature, dim=1),
            torch.softmax(teacher_logits / self.temperature, dim=1)
        ) * (self.temperature ** 2)
        student_loss = self.criterion_ce(student_logits, labels)
        return self.alpha * distillation_loss + (1 - self.alpha) * student_loss

criterion = DistillationLoss(temperature=3.0, alpha=0.5)

# Define optimizer and scheduler
optimizer = optim.Adam(student_model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=10)

# GPU power tracking
def get_gpu_power():
    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=power.draw', '--format=csv,nounits,noheader'],
            stdout=subprocess.PIPE, text=True
        )
        power_draws = [float(p) for p in result.stdout.strip().split('\n')]
        return sum(power_draws) / len(power_draws) if power_draws else 0.0
    except Exception as e:
        print(f"Error getting GPU power: {e}")
        return 0.0

# Energy tracking
tracker = EmissionsTracker()
tracker.start()

# Training loop
epochs = 25
gpu_power_readings = []
start_training_time = time.time()

for epoch in range(epochs):
    student_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Teacher predictions
        with torch.no_grad():
            teacher_logits = teacher_model(inputs)

        # Student predictions and loss computation
        student_logits = student_model(inputs)
        loss = criterion(student_logits, teacher_logits, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        gpu_power_readings.append(get_gpu_power())

    scheduler.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

end_training_time = time.time()
training_time = end_training_time - start_training_time

emissions = tracker.stop()

# Energy metrics
avg_gpu_power = sum(gpu_power_readings) / len(gpu_power_readings) if gpu_power_readings else 0.0
total_energy_consumption = (avg_gpu_power * training_time) / 3600  # kWh

# Model analysis
param_count = sum(p.numel() for p in student_model.parameters() if p.requires_grad)
torch.save(student_model.state_dict(), "student_model_vgg11_temp.pth")
model_size_mb = os.path.getsize("student_model_vgg11_temp.pth") / (1024 * 1024)

# FLOPs and inference time
example_input = torch.randn(1, 3, 32, 32).to(device)
flops, _ = profile(student_model, inputs=(example_input,), verbose=False)

student_model.eval()
with torch.no_grad():
    start_time = torch.cuda.Event(enable_timing=True)
    end_time = torch.cuda.Event(enable_timing=True)
    start_time.record()
    for _ in range(100):
        _ = student_model(example_input)
    end_time.record()
    torch.cuda.synchronize()
inference_time_ms = start_time.elapsed_time(end_time) / 100  # Average in milliseconds

# Output results
print("\n--- Model Analysis ---")
print(f"Parameter Count: {param_count}")
print(f"Model Size: {model_size_mb:.2f} MB")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Average Inference Time: {inference_time_ms:.6f} ms")

print("\n--- Energy and Emissions Report ---")
print(f"CO2 Emissions (CodeCarbon): {emissions:.6f} kg")
print(f"Average GPU Power Consumption: {avg_gpu_power:.2f} W")
print(f"Total GPU Energy Consumption: {total_energy_consumption:.6f} kWh")

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-6-03598a6d5a61>:53: F

Epoch [1/25], Loss: 6.3814


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[codecarbon INFO @ 03:56:55] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:56:55] Energy consumed for all GPUs : 0.001387 kWh. Total GPU Power : 65.97628938453347 W
[codecarbon INFO @ 03:56:55] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:56:55] 0.002371 kWh of electricity used since the beginning.
[codecarbon INFO @ 03:57:10] Energy consumed for RAM : 0.000119 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:57:10] Energy consumed fo

Epoch [2/25], Loss: 5.4015


...

Epoch [23/25], Loss: 1.6102


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[codecarbon INFO @ 04:23:40] Energy consumed for RAM : 0.002217 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 04:23:40] Energy consumed for all GPUs : 0.031100 kWh. Total GPU Power : 65.88136152456835 W
[codecarbon INFO @ 04:23:41] Energy consumed for all CPUs : 0.019838 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 04:23:41] 0.053155 kWh of electricity used since the beginning.
[codecarbon INFO @ 04:23:41] 0.011031 g.CO2eq/s mean an estimation of 347.870741927623 kg.CO2eq/year
[codecarbon INFO @ 04:23:55] Energy consumed for

Epoch [24/25], Loss: 1.4790


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[codecarbon INFO @ 04:24:56] Energy consumed for RAM : 0.002316 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 04:24:56] Energy consumed for all GPUs : 0.032488 kWh. Total GPU Power : 66.04941384539804 W
[codecarbon INFO @ 04:24:56] Energy consumed for all CPUs : 0.020724 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 04:24:56] 0.055528 kWh of electricity used since the beginning.
[codecarbon INFO @ 04:25:11] Energy consumed for RAM : 0.002335 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 04:25:11] Energy consumed fo

Epoch [25/25], Loss: 1.3560

--- Model Analysis ---
Parameter Count: 128807306
Model Size: 491.37 MB
FLOPs: 0.27 GFLOPs
Training Time: 1820.96 seconds
Average Inference Time: 3.505623 ms

--- Energy and Emissions Report ---
CO2 Emissions (CodeCarbon): 0.020110 kg
Average GPU Power Consumption: 69.05 W
Total GPU Energy Consumption: 34.928172 kWh


In [ ]:
#print(model)
!pip install ptflops

In [ ]:
# Test phase
student_model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = student_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f'Accuracy of the model on the test set: {test_accuracy:.2f}%')


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Accuracy of the model on the test set: 81.86%


In [ ]:
model = student_model

In [ ]:
# Test phase
model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f'Accuracy of the model on the test set: {test_accuracy:.2f}%')


Accuracy of the model on the test set: 81.86%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets, transforms
import torch.nn.utils.prune as prune
from ptflops import get_model_complexity_info
import numpy as np
from tqdm import tqdm
import time
import os
import subprocess
import codecarbon

# Initialize CodeCarbon tracker
tracker = codecarbon.EmissionsTracker()
tracker.start()

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the transforms for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 dataset
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Load the pretrained VGG16 model and modify the classifier
# model = models.vgg16(pretrained=True)
# input_last_layer = model.classifier[6].in_features
# model.classifier[6] = nn.Linear(input_last_layer, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Function to apply L1 unstructured pruning
def apply_pruning(model, amount=0.90):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)

# Function to remove pruning masks and make pruning permanent
def remove_pruning(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            if hasattr(module, 'weight_mask'):
                prune.remove(module, 'weight')
    return model

# Apply pruning to the model
apply_pruning(model, amount=0.90)

# Measure FLOPs using ptflops
def calculate_flops(model):
    model.eval()
    flops, params = get_model_complexity_info(model, (3, 32, 32), as_strings=False, print_per_layer_stat=False)
    return flops, params

# Measure inference time
def measure_inference_time(model, dataloader):
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)
            _ = model(images)
    end_time = time.time()
    return (end_time - start_time) / len(dataloader)

# Count non-zero parameters after pruning
def count_non_zero_params(model):
    non_zero_count = 0
    for param in model.parameters():
        if param.requires_grad:
            non_zero_count += torch.count_nonzero(param).item()
    return non_zero_count

# Training loop
train_start_time = time.time()
epochs = 15
for epoch in range(epochs):
    model.train()
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

train_end_time = time.time()
training_time = (train_end_time - train_start_time) / 60

# Remove pruning masks and make pruning permanent
model = remove_pruning(model)

# Calculate FLOPs, parameter count, and inference time after pruning
flops, params = calculate_flops(model)
param_count = count_non_zero_params(model)
inference_time = measure_inference_time(model, test_loader)

# Stop CodeCarbon tracker
emissions = tracker.stop()

# Save the pruned model
torch.save(model.state_dict(), 'pruned_vgg16_model.pth')

# Calculate model size
model_size = os.path.getsize('pruned_vgg16_model.pth') / (1024 * 1024)

# Print the final results
print("\n--- Model Analysis After Pruning ---")
print(f"Non-zero Parameter Count: {param_count}")
print(f"Model Size: {model_size:.2f} MB")
print(f"Training Time: {training_time:.2f} minutes")
print(f"Inference Time per Image: {inference_time:.6f} seconds")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"CO2 Emissions: {emissions:.6f} kg")


[codecarbon INFO @ 04:26:50] [setup] RAM Tracking...
[codecarbon INFO @ 04:26:50] [setup] GPU Tracking...
[codecarbon INFO @ 04:26:50] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 04:26:50] [setup] CPU Tracking...
[codecarbon WARNING @ 04:26:50] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 04:26:51] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 04:26:51] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 04:26:51] >>> Tracker's metadata:
[codecarbon INFO @ 04:26:51]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 04:26:51]   Python version: 3.10.12
[codecarbon INFO @ 04:26:51]   CodeCarbon version: 2.7.4
[codecarbon INFO @ 04:26:51]   Available RAM : 12.675 GB
[codecarbon INFO @ 04:26:51]   CPU count: 2
[codeca

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/15 - Training:  32%|███▏      | 200/625 [00:12<00:26, 15.91it/s][codecarbon INFO @ 04:27:06] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 04:27:06] Energy consumed for all GPUs : 0.000270 kWh. Total GPU Power : 64.71666736682052 W
[codecarbon INFO @ 04:27:06] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 04:27:06] 0.000467 kWh of electricity used since the beginning.
Epoch 1/15 - Training:  70%|██████▉   | 436/625 [00:27<00:11, 15.91it/s][code


--- Model Analysis After Pruning ---
Non-zero Parameter Count: 12890591
Model Size: 491.37 MB
Training Time: 9.89 minutes
Inference Time per Image: 0.025262 seconds
FLOPs: 0.27 GFLOPs
CO2 Emissions: 0.006783 kg


In [ ]:
print(model.features[0].weight)
print(model.features[3].weight)
#print(model.features[2].weight)
#print(model.features[5].weight)

Parameter containing:
tensor([[[[ 0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000]],

         [[-0.0000,  0.4803,  0.0000],
          [ 0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000,  0.0000]],

         [[ 0.1471,  0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.6428]]],


        [[[-0.8888, -0.0000, -0.0000],
          [ 0.7784,  0.0000,  0.0000],
          [-0.0000,  0.0000, -0.0000]],

         [[-0.0000,  0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000]],

         [[ 0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000]],

         [[-0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000]],

         [[ 0.0000,  0

In [ ]:
# Test phase
model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f'Accuracy of the model on the test set: {test_accuracy:.2f}%')


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Accuracy of the model on the test set: 73.71%
